In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc


In [2]:
fake = faker.Faker("ru_RU")

In [3]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 32
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE * 2
BPTree.HALF_LEAF_SIZE = 32
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE * 2

In [4]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class Data(msgspec.Struct):
    data: Any

In [5]:
index = BPTree[MyKey, Data]()

In [6]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [7]:
data = []
gc.collect()

340

In [8]:
%%time
for i in tqdm(range(10_000_000)):
    val = fake.name() + " - " + str(i)
    if index.insert(MyKey(val)) is None:
        print(f"Duplicate key: {val}")
    data.append(val)

  0%|          | 0/10000000 [00:00<?, ?it/s]

CPU times: user 8min 35s, sys: 5.92 s, total: 8min 41s
Wall time: 8min 39s


In [9]:
index.validate(), gc.collect()

({}, 18)

In [14]:
len(data), len(data) / (60 * 8 + 41)

(10000000, 19193.857965451054)

In [15]:
index.print_node(index.tree[index.root_node])


node-idx:179756, prev:None, next:None, level: 4  <============ *ROOT*
            |-> MyKey(full_name='Майя Робертовна Миронова - 474'): 3224, 179755


In [16]:
index.print_node(index.tree[3224])


node-idx:3224, prev:None, next:179755, level: 3 
         |-> MyKey(full_name='Агата Кузьминична Фокина - 28887'): 68, 164615
         |-> MyKey(full_name='Азарий Архипович Сазонов - 53200'): 164615, 86428
         |-> MyKey(full_name='Александр Богданович Тимофеев - 36711'): 86428, 183869
         |-> MyKey(full_name='Алина Натановна Киселева - 1283'): 183869, 46041
         |-> MyKey(full_name='Анастасия Харитоновна Нестерова - 5122'): 46041, 187738
         |-> MyKey(full_name='Анжела Ивановна Красильникова - 12110'): 187738, 99426
         |-> MyKey(full_name='Анисимова Майя Михайловна - 62705'): 99426, 198609
         |-> MyKey(full_name='Антонов Януарий Филимонович - 1145'): 198609, 24260
         |-> MyKey(full_name='Архип Феодосьевич Евсеев - 5994'): 24260, 183515
         |-> MyKey(full_name='Баранов Порфирий Тимурович - 9540'): 183515, 92506
         |-> MyKey(full_name='Беляева Фаина Степановна - 141362'): 92506, 179754
         |-> MyKey(full_name='Бобров Аристарх Ануфриев

In [17]:
index.print_node(index.tree[179755])


node-idx:179755, prev:3224, next:None, level: 3 
         |-> MyKey(full_name='Майя Робертовна Миронова - 474'): 103264, 3223
         |-> MyKey(full_name='Марина Вениаминовна Самойлова - 4393'): 3223, 133937
         |-> MyKey(full_name='Марфа Максимовна Назарова - 38618'): 133937, 70929
         |-> MyKey(full_name='Мечислав Зиновьевич Кондратьев - 12594'): 70929, 139938
         |-> MyKey(full_name='Михаил Богданович Филатов - 3164'): 139938, 31764
         |-> MyKey(full_name='Моисеев Аникита Власович - 174545'): 31764, 214937
         |-> MyKey(full_name='Морозова Надежда Владиславовна - 14467'): 214937, 115772
         |-> MyKey(full_name='Наина Кирилловна Копылова - 1642'): 115772, 60712
         |-> MyKey(full_name='Нестерова Надежда Даниловна - 14881'): 60712, 131464
         |-> MyKey(full_name='Никонов Трофим Фомич - 3809'): 131464, 15080
         |-> MyKey(full_name='Нинель Эдуардовна Фомичева - 60762'): 15080, 219385
         |-> MyKey(full_name='Носов Савватий Юльевич - 